# Step-4 Notebook for training SVM and MLP models 

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import os

### Load the training, validation and testing data

In [2]:
import pickle
processed_training_data_path = "./flowcam_split_data/plankton_data_101x64_final.pkl"
with open(processed_training_data_path, "rb") as file:
    trainAttrX, valAttrX, testAttrX, trainImagesX, \
        valImagesX, testImagesX, y_train, y_val, y_test = pickle.load(file)

In [3]:
y=[np.where(r==1)[0][0] for r in y_train]

In [4]:
yv=[np.where(r==1)[0][0] for r in y_val]

In [5]:
yt=[np.where(r==1)[0][0] for r in y_test]

### Load and train SVM model with different kernel types:

In [6]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(trainAttrX)
X_test = scaler.transform(valAttrX)

# Initialize the SVM classifier with One-vs-Rest strategy
svm_model = SVC(kernel='linear', C=1.0, random_state=42, decision_function_shape='ovr')
# svm_rbf_scale = SVC(kernel='rbf', gamma=.3, C=1.0, random_state=42)

# Train the model
svm_model.fit(X_train, y)

# Predict the test data
y_pred = svm_model.predict(X_test)

# Print the accuracy and classification report
print("Accuracy:", accuracy_score(yv, y_pred))
print("Classification Report:\n", classification_report(yv, y_pred, zero_division=1))

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         7
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         8

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



### Save the classification report to results directory

In [7]:
report_dict = classification_report(yv, y_pred, zero_division=1,output_dict=True)
report_df = pd.DataFrame.from_dict(report_dict).transpose()

In [8]:
!pip install openpyxl

In [9]:
os.makedirs('./results/svc',exist_ok=True)
report_df.to_excel("./results/svc/val_report_svc_linear.xlsx")
report_df=pd.read_excel("./results/svc/val_report_svc_linear.xlsx")

In [10]:
sum(yv==y_pred)/len(y_pred)

1.0

In [11]:
report_df

,Unnamed: 0,precision,recall,f1-score,support
0,0,1,1,1,6
1,1,1,1,1,7
2,2,1,1,1,2
3,3,1,1,1,8
4,accuracy,1,1,1,1
5,macro avg,1,1,1,23
6,weighted avg,1,1,1,23


# MLP Model

In [12]:
import numpy as np
import pickle
import os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.initializers import glorot_uniform, Constant #, HeNormal, Zeros,
from sklearn.utils import class_weight
import matplotlib.pyplot as plt

kernel_init = glorot_uniform()
bias_init = Constant(value=0.1)


### Define MLP model with path to trained weights 

In [13]:
def multilayer_perceptron(dim: int, num_classes: int, activation: str = "relu") -> Model:
    model = Sequential()

    # Input layer
    model.add(Dense(1024, input_dim=dim, activation=activation, kernel_initializer=kernel_init, bias_initializer=bias_init))
    model.add(BatchNormalization())

    # Hidden layers
    for _ in range(1):
        model.add(Dense(512, activation=activation, kernel_initializer=kernel_init, bias_initializer=bias_init))
        model.add(BatchNormalization())

    model.add(Dropout(0.4))

    # Output layer
    model.add(Dense(num_classes, activation="softmax"))
    model.build((None, dim))
    model.summary()

    return model

def train_mlp(path_to_split_data,trained_model_folder,epochs=10) -> None:
    processed_training_data_path = path_to_split_data
    with open(processed_training_data_path, "rb") as file:
        trainAttrX, valAttrX, testAttrX, trainImagesX, valImagesX, testImagesX, y_train, y_val, y_test = pickle.load(file)

    model = multilayer_perceptron(trainAttrX.shape[1], y_train.shape[1])

    trained_path = trained_model_folder
    os.makedirs(trained_path,exist_ok=True)
    output_path = os.path.join(trained_path, "mlp_model_flowcam_data.h5")

    # Save the best model so far when training
    checkpoint = ModelCheckpoint(output_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    # Lower learning rate when models learning has plateaued
    lr_drop = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.000001)
    # Stop training if signs of model convergence
    early_stopping = EarlyStopping(monitor='loss', patience=80)
    # Enables us to start tensorboard from console for diagnostic tools
    # tensor_board = TensorBoard(log_dir='Graph_mlp', histogram_freq=0, write_graph=True, write_images=True)
    callbacks_list = [checkpoint, lr_drop, early_stopping]#, tensor_board]

    y = [np.where(r == 1)[0][0] for r in y_train]
    for v in np.where(~y_train.any(axis=0))[0]:
        for _ in range(1000):
            y.append(v)
    classes = np.unique(y)
    class_weights = class_weight.compute_class_weight('balanced', classes=classes, y=y)
    class_weights = dict(zip(classes, class_weights))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(trainAttrX, y_train, epochs=epochs, validation_data=(valAttrX, y_val), callbacks=callbacks_list, batch_size=500, class_weight=class_weights)

    # with open(os.path.join(trained_path, "{}_history".format("mlp")), 'wb') as file:
    #     pickle.dump(history.history, file)

    return history


### Train the MLP model

In [15]:
# Run the training function
processed_training_data_path = "./flowcam_split_data/plankton_data_101x64_final.pkl"
trained_model_folder = "./trained_models"
hist=train_mlp(processed_training_data_path,trained_model_folder,epochs=4)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1024)              45056     
                                                                 
 batch_normalization_2 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_3 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 4)                